# Sistemas Inteligentes 2021/2022

## Mini-projeto 3: Aprendizagem Automática

## Relatório


## Grupo: 51

Número: 56926    Nome: Lucas Pinto   
Número: 56895    Nome: Matilde Silva    
Número: 56941    Nome: Bruno Gonzalez

## 1. Processamento dos dados

(remover) Descrever e comentar os passos na leitura e processamento dos dados 

In [23]:
import pandas as pd
import numpy as np

df_cirrose = pd.read_csv("train.csv")

col_idxs = [x for x in range(1, 19)]
col_names = np.array(df_cirrose.columns)[col_idxs]

X = df_cirrose.values[:, col_idxs]
y = df_cirrose.values[:, 19]
y = y.astype('int')

dummy_col_names = ['Status', 'Drug', 'Sex',
                   'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

df_cirrose_X = df_cirrose[col_names]
df_cirrose_Xdum = pd.get_dummies(df_cirrose_X, columns=dummy_col_names)
col_names_dum = np.array(df_cirrose_Xdum.columns)

Xdum = df_cirrose_Xdum.values


## 2. Ajustamento dos modelos


### 2.1. Teste de modelos e validação

(remover) Enunciar as várias abordagens testadas para
* Criar modelos
* Validar modelos
* Avaliar a qualidade dos modelos

In [24]:
from sklearn.tree import DecisionTreeClassifier

sk_dt = DecisionTreeClassifier(criterion="entropy")
sk_dt.fit(Xdum, y)

preds = sk_dt.predict(Xdum)
for i in range(df_cirrose_Xdum.shape[0]):
    print(f"Classe real: {y[i]}; Previsão: {preds[i]} ")

print("\nNúmero Total de más predições: ", sum(preds != y))


Classe real: 1; Previsão: 1 
Classe real: 2; Previsão: 2 
Classe real: 1; Previsão: 1 
Classe real: 2; Previsão: 2 
Classe real: 3; Previsão: 3 
Classe real: 4; Previsão: 4 
Classe real: 3; Previsão: 3 
Classe real: 4; Previsão: 4 
Classe real: 4; Previsão: 4 
Classe real: 4; Previsão: 4 
Classe real: 4; Previsão: 4 
Classe real: 1; Previsão: 1 
Classe real: 1; Previsão: 1 
Classe real: 3; Previsão: 3 
Classe real: 3; Previsão: 3 
Classe real: 1; Previsão: 1 
Classe real: 3; Previsão: 3 
Classe real: 4; Previsão: 4 
Classe real: 4; Previsão: 4 
Classe real: 4; Previsão: 4 
Classe real: 1; Previsão: 1 
Classe real: 2; Previsão: 2 
Classe real: 2; Previsão: 2 
Classe real: 2; Previsão: 2 
Classe real: 3; Previsão: 3 
Classe real: 2; Previsão: 2 
Classe real: 4; Previsão: 4 
Classe real: 4; Previsão: 4 
Classe real: 4; Previsão: 4 
Classe real: 4; Previsão: 4 
Classe real: 3; Previsão: 3 
Classe real: 1; Previsão: 1 
Classe real: 4; Previsão: 4 
Classe real: 4; Previsão: 4 
Classe real: 3

### 2.2. Seleção e apresentação do melhor modelo

(remover) Apresentar todos os resultados para cada modelo obtido

* Elencar o critério de selecção
* Apresentar a árvore (se aplicável)
* Discutir os resultados


## 3. Validação do modelo final ajustado com um conjunto de validação independente

(remover) O ficheiro `test.csv` contém um conjunto de dados não usado para treinar o modelo definido no ponto #2. 
* Preparar o data set
* Correr o modelo selecionado como sendo o melhor nestes dados
* Apresentar os resultados para as métricas de validação
* Discutir os resultados alcançados confrontando os resultados com os obtidos anteriormente

## 4. Competição

(remover) Explicar os passos feitos para chegar à solução submetida na competição, tendo em conta:

* A escolha dos parâmetros usados pelos classificadores
* A escolha de atributos mais relevantes para a aprendizagem dos modelos
    

In [32]:
df_cirrose_test = pd.read_csv("test.csv")

col_idxs = [x for x in range(1, 19)]
col_names = np.array(df_cirrose_test.columns)[col_idxs]

dummy_col_names = ['Status', 'Drug', 'Sex',
                   'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

df_cirrose_test_X = df_cirrose_test[col_names]
df_cirrose_test_Xdum = pd.get_dummies(df_cirrose_test_X, columns=dummy_col_names)

Xdum = df_cirrose_test_Xdum.values
preds = sk_dt.predict(Xdum)


# # list(zip(df_cirrose_Xdum.shape, preds))
df_out = pd.DataFrame({'ID': df_cirrose_test['ID'], 'Stage': preds})

df_out.to_csv('submission.csv', index=False)